In [7]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import cv2
import numpy as np
from loadimages import load_test_images
import random
from models import FlowNetVector, FlowNetS, FlowNetS4
from matplotlib import pyplot as plt

In [8]:
sequences_s = load_test_images(1)
sequences_s4 = load_test_images(3)
sequences_v = load_test_images(1)

In [9]:
print(len(sequences_v))

18


In [10]:
mse_loss = nn.MSELoss()
avg_pool = nn.AvgPool2d(2)
upsample_16 = nn.Upsample(scale_factor=16, mode='nearest')
upsample_32 = nn.Upsample(scale_factor=32, mode='nearest')
upsample_64 = nn.Upsample(scale_factor=64, mode='nearest')
upsample_128 = nn.Upsample(scale_factor=128, mode='nearest')

In [11]:
def get_block_vector(block1, block2, block_size):
  x_y = [0, 0]
  min_error = float('inf')
  for i in range(2 * block_size):
    for j in range(2 * block_size):
      error = mse_loss(block1, block2[:,i:i + block_size, j:j + block_size])
      if error < min_error:
        min_error = error
        x_y = [block_size - i - 1, block_size - j - 1]
  return x_y

In [12]:
def get_vector_layer(image1, image2, size):
  vectors = np.zeros(size)
  for j in range(1, size[0] + 1):
    for k in range(1, size[1] + 1):
      x_y = get_block_vector(
        image1[:,j * block_size: j * block_size + block_size,k * block_size: k * block_size + block_size],
        image2[:,(j - 1) * block_size: (j + 2) * block_size,(k - 1) * block_size: (k + 2) * block_size],
        block_size
      )
      vectors[j - 1][k - 1][0] = x_y[0]
      vectors[j - 1][k - 1][1] = x_y[1]
  return vectors

In [13]:
def downsize_images(image1, image2):
  image1 = avg_pool(image1)
  image2 = avg_pool(image2)
  return (image1, image2)

In [14]:
block_size = 16
def find_motion_images(image1, image2):
  first_layer_vector_image = get_vector_layer(image1, image2, (30, 22, 2))

  (image1, image2) = downsize_images(image1, image2)
  second_layer_vector_image = get_vector_layer(image1, image2, (14, 10, 2))

  (image1, image2) = downsize_images(image1, image2)
  third_layer_vector_image = get_vector_layer(image1, image2, (6, 4, 2))

  (image1, image2) = downsize_images(image1, image2)
  fourth_layer_vector_image = get_vector_layer(image1, image2, (2, 1, 2))
  
  img_f1_x = upsample_16(torch.tensor(np.pad(first_layer_vector_image[:,:,0], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f1_y = upsample_16(torch.tensor(np.pad(first_layer_vector_image[:,:,1], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f2_x = upsample_32(torch.tensor(np.pad(second_layer_vector_image[:,:,0], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f2_y = upsample_32(torch.tensor(np.pad(second_layer_vector_image[:,:,1], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f3_x = upsample_64(torch.tensor(np.pad(third_layer_vector_image[:,:,1], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f3_y = upsample_64(torch.tensor(np.pad(third_layer_vector_image[:,:,1], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f4_x = upsample_128(torch.tensor(np.pad(fourth_layer_vector_image[:,:,0], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)
  img_f4_y = upsample_128(torch.tensor(np.pad(fourth_layer_vector_image[:,:,1], ((1, 1), (1, 1)), mode='constant')).unsqueeze(0).unsqueeze(0).type(torch.FloatTensor)).squeeze(0)

  return  (img_f1_x, img_f1_y, img_f2_x, img_f2_y, img_f3_x, img_f3_y, img_f4_x, img_f4_y)

In [15]:
vectors = list()
for sequence in sequences_v:
  (img_f1_x, img_f1_y, img_f2_x, img_f2_y, img_f3_x, img_f3_y, img_f4_x, img_f4_y) = find_motion_images(sequence[0][:3], sequence[0][3:6])
  print(img_f1_x.shape)
  vectors.append([
    torch.cat((
      sequence[0], img_f1_x, img_f1_y, img_f2_x, img_f2_y, img_f3_x, img_f3_y, img_f4_x, img_f4_y
    ), 0).type(torch.FloatTensor),
    sequence[1]])

torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])
torch.Size([1, 512, 384])


In [16]:
model_s = FlowNetS()
model_s.load_state_dict(torch.load('./flownets.pt', map_location=torch.device('cpu')))

model_s4 = FlowNetS4()
model_s4.load_state_dict(torch.load('./flownets4.pt', map_location=torch.device('cpu')))

model_v = FlowNetVector()
model_v.load_state_dict(torch.load('./vectornet.pt', map_location=torch.device('cpu')))
#model.cuda()

<All keys matched successfully>

In [18]:
for i in range(len(sequences_s)): # change this to sequences
  test_img = cv2.resize(sequences_s[i][1].cpu().permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
    
  out = model_v(vectors[i][0].unsqueeze(0)).squeeze(0).cpu()
  vector_img = cv2.resize(out.permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
  vector_difference = vector_img - test_img
  
  out = model_s(sequences_s[i][0].unsqueeze(0)).squeeze(0).cpu()
  s_img = cv2.resize(out.permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
  s_difference = s_img - test_img
    
  out = model_s4(sequences_s4[i][0].unsqueeze(0)).squeeze(0).cpu()
  s4_img = cv2.resize(out.permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
  s4_difference = s4_img - test_img

  first = cv2.resize(sequences_s[i][0][:3].cpu().permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
  second = cv2.resize(sequences_s[i][0][3:6].cpu().permute(1, 2, 0).detach().numpy(), dsize=(192, 256), interpolation=cv2.INTER_CUBIC)
  
  cv2.imwrite('before' + str(i) + '.jpg', first)
  cv2.imwrite('after' + str(i) + '.jpg', second)
    
  cv2.imwrite('generated vector' + str(i) + '.jpg', vector_img)
  cv2.imwrite('generated vector difference' + str(i) + '.jpg', vector_difference)
    
  cv2.imwrite('generated s' + str(i) + '.jpg', s_img)
  cv2.imwrite('generated s difference' + str(i) + '.jpg', s_difference)

  cv2.imwrite('generated s4' + str(i) + '.jpg', s4_img)
  cv2.imwrite('generated s4 difference' + str(i) + '.jpg', s4_difference)
    
  cv2.imwrite('test' + str(i) + '.jpg', test_img)